In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter, WeekdayLocator,DayLocator, MONDAY
#from  matplotlib.finance  import  candlestick_ohlc
import mpl_finance 

class CSymbol():                
    def __init__(self,specs={}):
                
        #get symbols specifications like swaps min volum max volum symbol name precision contract volum etc..
        self.specs=specs            
        self.rates=pd.read_csv(specs["file_name"],sep="\t")[0:5] #only 5 for easy debug for now...
        
        ##delete this just for adebug i forced delete some items to reproduce the syncronization issues
        if specs["symbol_name"]=="GBPUSD":
            self.rates.drop(0,inplace=True)
            
        if specs["symbol_name"]=="USDJPY":
            self.rates.drop(2,inplace=True)
            
        if specs["symbol_name"]=="EURUSD":
            self.rates.drop(3,inplace=True)            
            
        self.rates.columns=specs["symbol_name"]+self.rates.columns        
        self.rates = self.rates.rename(columns={(specs['symbol_name']+"Datetime"):"time"})                
        self.rates.set_index("time",inplace=True)
        return None
    
    #TODO if we whant a live reader enviroment 
    def updaterates():
        return None
    
#Not clear now    
class CTrade():
    
    def __init__(self,simulator):
        self.simulator=simulator
        return None
    def openlong(self,symbol,volum):
        return None
    def openshort(self,symbol,volum):
        return None
#    
class CPositionInfo():
    
    
    def __init__(self,volumn,xtype,symbol,step_open):
        self.volumn=0.
        self.type=-1 #0 short  1 long
        self.symbol = None#class symbol
        self.step_open =None#step open price
        self.step_close = None #none or step to close price
        self.current_profit = 0. #positive or nevative profit
        
        return None
    
    
#define custom strategy...
class CStrategy():    
    def __init__(self,name,simulator):
        self.name="Custom strategy name"
        self.simulator=simulator
        return None
    
    def run_strategy(self,model=None):
        while self.simulator.Turn():
            #do somthing indicator working then
            operation_list=[]
            simulator.Perform_OPS(Operations_list)            
        return None
    
#same as CStrategy but multiple strategy with shared simulator    
class CMultiStrategy():
    def __init__(self,strategy_list,simulator):
        return None

class CIndicatorMA():
    def __init__(self,symbol,name="MAInd"):
        self.symbol=symbol
        self.name=name
        return None
    def calculation():
        
        return None
    def pltInd():
        return None
    
class CIndicatorMulti():
    def __init__(self,symbols=[],name="CutomMultiIndi"):
        self.symbols=symbols #list of all symbols sycronized
        self.name=name
        self.fast=5
        self.slow=10
        return None
    def calculation():
        
        return None
    def plotInd():
        return None
class CSimulator:    
    
    def __init__(self,enviroment,i_symbols=[],inital_step=0):        
        self.Symbols=[] 
        self.PosInfo=[]#list of active positions
        self.PosHist=[]#history all closed positions
        self.StatsHist=[]#history all stats
        self.enviroment=enviroment
        #basic stats
        self.initial_balance=enviroment["balance"]
        self.current_balance=self.initial_balance
        self.current_equity=self.initial_balance
        #TODO max min equity balance max min equi balanc drwadown etc..
        for x in i_symbols:
            newsym=CSymbol(x)
            self.Symbols.append(newsym)
            
        
        self.syncronize_symbols()#syncronize all symbols indexs
        self.set_cursor(inital_step)#init cursos at step 0
        return None
    
    #TODO syncronize all symbols datetimes
    def syncronize_symbols(self):
        newindex=""
        for x in range(len(self.Symbols)-1):
            newindex= self.Symbols[x].rates.index.union(self.Symbols[x+1].rates.index)

        for x in range(len(self.Symbols)):
            self.Symbols[x].rates=self.Symbols[x].rates.reindex(newindex)
            
        for x in range(len(self.Symbols)):            
            self.Symbols[x].rates.fillna(method='ffill',inplace=True)
           
        
        #once we filled all na values with previous value only na values apear if first time step is na
        #we delete this time step and previus for all time series
        natimes=[]
        for x in range(len(self.Symbols)):
            natimes.append(self.Symbols[x].rates[self.Symbols[x].rates.isnull().any(axis=1)].index.tolist() )
                
        
        maxtimevalid=max(natimes)[0]
        
        for x in range(len(self.Symbols)):
            tmp=self.Symbols[x].rates.reset_index()
            self.Symbols[x].rates=tmp[tmp["time"]>maxtimevalid]
            self.Symbols[x].rates.set_index("time",inplace=True)
                
        return None
    
    def plotSimulator(indicators=False):
        #for each symbol on Symbols plot until cursor
        
        return None
    
    #if we whant interactive session we set or initialize cursor at step we needed
    def set_cursor(self,step=0):
        self.current_step=step
        
        return None
    
    #for interactive session return true while step < len(rates)
    #example usage while simulator.Turn():
    #                    perform_ops...    
    def Turn(self):
        #step +=1
        self.calcSates()
        return False
    
    #use together with Turn
    def Perform_OPS(operation):
        return None
    #return n pandas frames with all Symbols sycronizeds
    def getRatesHist(self):
        ret=pd.DataFrame()
        lista=[]
        for x in range(len(self.Symbols)):
            ret=pd.concat((ret,self.Symbols[x].rates),axis=1)
            lista.append(self.Symbols[x])
        return ret,lista
    
    #return current stadisitical opbjects
    def getStats(self):
        return None
    
    #return all current stadistical object from step 0 to current state
    def getStatsHistory(self):
        return None
    
    #private method called at end of turn
    def calcSates(self):
        return None
    
    #return all active positions
    def PositionsCount():
        return None
    

    
def main():
    all_symbols=[]
    specs={"file_name":'data/EURUSD0.csv',
           "symbol_name":"EURUSD",
           "precision":5,
           "swap_long":-0.8,
           "swap_short":-0.4,
           "contract_volum":100000}
    all_symbols.append(specs)
    
    specs={"file_name":'data/GBPUSD0.csv',
           "symbol_name":"GBPUSD",
           "precision":5,
           "swap_long":-0.8,
           "swap_short":-0.4,
           "contract_volum":100000}
            
    
    all_symbols.append(specs)
    
    specs={"file_name":'data/USDJPY0.csv',
           "symbol_name":"USDJPY",
           "precision":3,
           "swap_long":-0.8,
           "swap_short":-0.4,
           "contract_volum":100000}
    
    all_symbols.append(specs)
    
    initial_step=0
    enviroment={"balance":10000}
    
    
    simulator=CSimulator(enviroment,all_symbols,initial_step)
    
    #vamos ha hacer la session interactiva y plotar el simulador sin indicadores
    
    #usage method 1 
    #once simulator initialited and symbols sycrnonized we can use them on our ML/DL model
    #we can uset getRatesHist to perform our models
    #then we can define a custom class stragey with
    #print(simulator.Symbols[1].rates.isnan())
    #print(simulator.Symbols[2].rates.isnan)
    #print(x_train.columns)
    #train you model operations
    #model=None
    #strategy.run_strategy(model)
    
    #or we can use on Q-learning or NEAT or genetic algorithm
    #strategy=CStrategy("strategyname",simulator)    
    #strategy.run_strategy()
    
    #strategy=CStrategy("strategyname",simulator)
    x_train,list_mode=simulator.getRatesHist() #return all rates synconized y pandas tabular and list_mode independent
            
    #lets go to try add some indicator and plot them
    #print(simulator.Symbols[0].rates)
    #print(simulator.Symbols[1].rates)
    #print(simulator.Symbols[2].rates)
    print(x_train,len(list_mode))
    
    indicator=CIndicatorMulti(list_mode,"ClusterIndicador")
    
    
    #after we fill all na with previous values we need all na values are related to first position we need
    #sincronize all timeseries with first position valid for all series 
    
    #print(simulator.Symbols[1].rates.isnull().any(axis=1))
    #print()
    
    #finally remove all null index from all timeseries

ModuleNotFoundError: No module named 'mpl_finance'

In [ ]:
main()